In [2]:
from pathlib import Path
from thermophysicalProperties import Database
import matplotlib.pyplot as plt
import numpy as np
from frozendict import frozendict
import pint
ureg = pint.UnitRegistry(auto_reduce_dimensions=True)
from uncertainties import ufloat
import os
from numdifftools import Derivative
from scipy.optimize import minimize
import warnings
import sys
import thermoToolsAdditions as tta
from thermoToolsAdditions import convert_to_thermochem_name

warnings.filterwarnings("ignore")

thermochimica_path = Path("../../thermochimica")
output_path = Path('../outputs')
output_name = 'output.json'
data_file = Path("../../thermochimica/data/MSTDB-TC_V3.0_Fluorides_No_Functions_8-2.dat")
input_file_name = "runThermochimica.ti"

# Get the script name from sys.argv
script_name = sys.argv[0]

# Get the absolute path of the script
script_dir = Path(os.path.abspath(script_name)).resolve().parent

mstdb_tp_path = Path('../../mstdb-tp/Molten_Salt_Thermophysical_Properties.csv')
mstdb_tp_rk_path = Path('../../mstdb-tp/Molten_Salt_Thermophysical_Properties_RK.csv')

db = Database(mstdb_tp_path, mstdb_tp_rk_path)

# Fuel Optimization

For the fuel salt, a chloride salt was chosen, and has been identified as a good choice for fast systems by a number of reactor designers (e.g. Terra Power) due to its relatively high $A$ (minimal moderation) and various fissile components and fission products tend to have high solubilities in chloride melts, making them good fuel carriers. The base components under consideration are
1. $\text{KCl}$
2. $\text{NaCl}$
3. $\text{ZrCl}_4$
4. $\text{AlCl}$
5. $\text{UCl}_3$ (the most common redox state of U in chloride melts)
6. $\text{PuCl}_3$ (also the most common redox state)
7. $\text{NdCl}_3$ (only used as a surrogate for Cm)

A notable exclusion is $\text{MgCl}_2$, which is often considered due to its ability to control redox or lower the melting point of the salt, which can be a concern for chlorides which generally have higher melting points than fluorides although $\text{RbCl}$ can also be used for this purpose, however, no thermophysical properties are currently available in MSTDB-TP, so it has been excluded. Additionally, it was determined that Zr additives would be used for redox control, so Mg is not necesssary. $\text{PuCl}$ and $\text{UCl}$ are both present in the salt, however we will consider their ratio fixed (these components are more relevant for neutronics than thermal hydraulics) and also the fraction of actinides in the fuel salt is also constrained by criticality and fuel cycle requirements. We will perform this calculation with a fixed actinide fraction with the hope that they're soluble at the periphery of the fuel tube, and we will verify this after the fact.

For the fuel salt, the heating requirements are also a concern, and we would like the salt to not have a prohibitively high melting point, but in this design, the fuel salt will actually reach very high temperatures on the centerline, and the boiling temperature of the fuel salt becomes a relevant consideration. To acheive a high power density in the salt (comparable to a LWR at ~100 kW/L) a boiling point of around 1800 K is desirable, based on rudimentary thermohydraulics scoping calculations.

Availability of thermophysical properties
| Pure Compound   | $\rho$ | $\mu$ | $k$  | $c_p$ |
| --------------- | ------ | ----- | ---- | ----- | 
| $\text{KCl}$    |   ✓    |   ✓   |  ✓   |   ✓   |
| $\text{NaCl}$   |   ✓    |   ✓   |  ✓   |   ✓   |
| $\text{ZrCl}_4$ |   ✓    |   ✓   |  ✗   |   ✗   |
| $\text{AlCl}$   |   ✓    |   ✓   |  ✗   |   ✓   |
| $\text{UCl}_3$  |   ✓    |   ✗   |  ✗   |   ✗   |
| $\text{PuCl}_3$ |   ✗    |   ✗   |  ✗   |   ✓   |
| $\text{NdCl}_3$ |   ✓    |   ✓   |  ✗   |   ✓   |

Which is admittedly quite sparse, but is more than sufficient to fully calculate all of the thermophysical properties of a salt of interest (so long as we don't exclude some endmembers).

## Optimization Methodology
An important consideration in the fuel sallt is the centerline temperature. In our design, since molten salt is a relatively poor conductor, the centerline can reach _very_ high temperatures, and the constraint then becomes the fuel salt melting point. The temperature profile (assuming pure conduction) is parabolic, and the centerline temperature is given by
$$
    T(0) = \frac{\dot{q}r_o^2}{4k} + T_\infty + \frac{\dot{q}r_o}{2h}\nonumber
$$
Where $T(0)$ is the centerline temperature, $\dot{q}$ is the volumetric heat generation rate within the rod (our target is around 100 kW/L), $r_o$ is the outer fuel rod radius, $k$ is the fuel salt thermal conductivity, $h$ is the heat transfer coefficient from the outer rod surface to the coolant salt (which is not affected by the fuel salt), and $T_\infty$ is the bulk coolant temperature away from the thermal boundary layer near the rod surface. From this expression, it's clear that increasing the thermal conductivity of the fuel salt and/or raising its boiling point will loosen the constraints on our design. However, we also want our fuel salt to melt at a reasonable temperature (say 800-900 K). So our goal will be to maximize the thermal conductivity while keeping the melting point below 800-900 K, and the boiling point above 2000 K.

## Actinide Composition
The fuel composition is (representing an actinide fraction of 55 mole %): $\text{PuCl}_3$ - $\text{NdCl}_3$ - $\text{UCl}_3$ ($39.8$ - $0.2$ - $15.0$ mole %) which is notably _mostly_ $\text{PuCl}_3$, the rest of the fuel salt (45 mole %) is unconstrained. 

In [ ]:
T_max = 900 # Maximum melting point
T_min = 2000

actinide_composition = np.array([0.398, 0.002, 0.15])

def check_melting_and_boiling(x, endmembers, elements_used):
    x = np.append(x, 0.45-sum(x))
    x = np.append(x, actinide_composition)

    # Convert from NaF to Na F (unfortunate mismatch in convention)
    new_endmembers = [convert_to_thermochem_name(endmember) for endmember in endmembers]
    salt = dict({endmember: x[i] for i, endmember in enumerate(new_endmembers)})

    try:
        T_m, T_b = tta.calculate_melting_and_boiling(thermochimica_path, output_path, output_name, data_file, salt, elements_used, \
                                                   suppress_output=True, phase_tolerance=0.9)
        return min(T_max - T_m, T_b - T_min)
    except:
        # Return a large negative value to indicate that the constriaint was not satisfied
        return -1E+06 # The supplied salt does not have a melting or boiling point in the given range (it could be that the salt sublimates)
    
# A constraint function which ensures that sum(x) = 1
def constraint(x):
    return 0.45 - np.sum(x)

# Define the constraints in a format that `minimize` can use
cons = ({'type': 'ineq', 'fun': constraint})

In [ ]:
T_f = 1350 # Assumed average fuel temperature in K

# Define the objective function
def objective(x, endmembers):
    x = np.append(x, 1-sum(x))
    x = np.append(x, actinide_composition)
    salt = frozendict({endmember: x[i] for i, endmember in enumerate(endmembers)})
    try:
        k = db.get_tp('thermal_conductivity', salt)(T_f).nominal_value
    except OverflowError:
        return 1E+03
    objective = k
    if np.isnan(objective) or np.isinf(objective):
        return 1E+03
    else:
        return objective

In [ ]:
endmembers = ['NaCl', 'KCl', 'ZrCl4', 'PuCl3', 'NdCl3', 'UCl3']
elements_used = ['Cl', 'Na', 'K', 'Zr', 'Pu', 'Nd', 'U']

# Define the constraints in a format that `minimize` can use
cons = ({'type': 'ineq', 'fun': constraint, 'tol': 1e-6}, \
        {'type': 'ineq', 'fun': check_melting_and_boiling, 'args': (endmembers, elements_used), 'tol': 1e-6})

In [1]:
test_composition = {'Na Cl': 0.3005, 'K Cl': 0.054, 'Zr Cl_4': 0.01, 'Pu Cl_3': 0.398, 'Nd Cl_3': 0.002, 'U Cl_3': 0.15}
tta.calculate_melting_and_boiling(thermochimica_path, output_path, output_name, data_file, \
                                             test_composition, elements_used, ntstep=150, phase_tolerance=0)

NameError: name 'tta' is not defined